In [10]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [11]:
import sys
#sys.path.append('/data/tjarniko/MEOPAR/mocsy')
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ')
sys.path.append('/data/tjarniko/MEOPAR/tools/SalishSeaTools/salishsea_tools/')
import mocsy
import river_201702 as rv
import VIZ_plots as VP

import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import seawater as sw
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt
from matplotlib import reload
plt.style.use('seaborn-bright')
import netCDF4 as nc
import numpy as np
import cmocean as cm

""
# from salishsea_tools import (
#     nc_tools,
#     viz_tools,
#     geo_tools,
#     tidetools
# )

%matplotlib inline

In [12]:
w2 = nc.Dataset('/data/tjarniko/MEOPAR/rivers-climatology/skog/rivers_skog_tracers_m12d31.nc')
print(w2)

low_dic = w2.variables['dic'][:]
low_dic[:] = 660
low_ta = w2.variables['ta'][:]
low_ta[:] = 630
low_dic[0,500,394] = 820
low_ta[0,500,394] = 750

#'Fraser': {'depth': 3, 'di': 1, 'dj': 1, 'i': 500, 'j': 394, 'prop': 0.984},

# w2.variables['dic'][0,500,394,]

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    history: Fri Aug 10 11:06:33 2018: ncks -4 -L4 -O rivers_skog_tracers_m12d31.nc rivers_skog_tracers_m12d31.nc
    NCO: 4.4.2
    dimensions(sizes): time_counter(1), y(898), x(398)
    variables(dimensions): float64 bsi(time_counter,y,x), float64 dia(time_counter,y,x), float64 dic(time_counter,y,x), float64 don(time_counter,y,x), float64 mes(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float64 nh4(time_counter,y,x), float64 no3(time_counter,y,x), float64 oxy(time_counter,y,x), float64 phy(time_counter,y,x), float64 pon(time_counter,y,x), float64 sil(time_counter,y,x), float64 ta(time_counter,y,x), float64 time_counter(time_counter), float64 tur(time_counter,y,x), float64 zoo(time_counter,y,x)
    groups: 



In [13]:

daylist = [31,29,31,30,31,30,31,31,30,31,30,31]
tstrlist = []
#/data/tjarniko/MEOPAR/rivers-climatology/skog/
#rivers_skog_tracers_m12d31.nc
for m in range(1,13):
    daymon = daylist[m-1]
    for d in range(1,daymon+1):
        #tdset = '/data/tjarniko/MEOPAR/rivers-climatology/skog/'
        mstr = 'm' + str(m)
        if m <10:
            mstr = 'm0'+str(m)
        dstr = 'd' + str(d)
        if d < 10:
            dstr = 'd0' + str(d)

        mdstr = mstr+dstr
        tstr = 'rivers_skog_tracers_' + mdstr + '.nc'
#         print(tstr)
        tstrlist.append(tstr)


In [14]:
for i in range(0, len(tstrlist)):
    print(i)
    tstr = tstrlist[i]
    tin = '/data/tjarniko/MEOPAR/rivers-climatology/skog/' + tstrlist[i]
#    print(tin)
    w = nc.Dataset(tin)
#     print(w)

    dsin = w

    #output file
    ncname = '/data/tjarniko/MEOPAR/rivers-climatology/skog_sens_highFras/' + tstrlist[i]
    dsout = nc.Dataset(ncname, "w", format="NETCDF4")
    for dname, the_dim in dsin.dimensions.items():
        #print(len(the_dim))
        dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
    for v_name, varin in dsin.variables.items():
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})

        outVar[:] = varin[:]

    dsout['dic'][:] = low_dic[:]
    dsout['ta'][:] = low_ta[:]
    dsout.close()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
# for i in range(0,len(tstrlist)):
#     print(i)
#     tstr = tstrlist[i]
#     tin = '/data/tjarniko/MEOPAR/rivers-climatology/skog/' + tstrlist[i]
# #    print(tin)
#     w = nc.Dataset(tin)
# #     print(w)

#     dsin = w

#     #output file
#     ncname = '/data/tjarniko/MEOPAR/rivers-climatology/skog_sens2/' + tstrlist[i]
#     dsout = nc.Dataset(ncname, "w", format="NETCDF4")
#     for dname, the_dim in dsin.dimensions.items():
#         print(dname)
# #             print(dname)
#         #print(len(the_dim))
#         dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
#     for v_name, varin in dsin.variables.items():
#         if ((v_name == 'dic') | (v_name == 'ta')):
#             #print(v_name)
#             outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
#             outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})

#             outVar[:] = varin[:]

#     dsout['dic'][:] = low_dic[:]
#     dsout['ta'][:] = low_ta[:]
#     dsout.close()


In [16]:
w = nc.Dataset('/data/tjarniko/MEOPAR/rivers-climatology/skog_sens_highFras/rivers_skog_tracers_m12d31.nc')
print(w)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time_counter(1), y(898), x(398)
    variables(dimensions): float64 bsi(time_counter,y,x), float64 dia(time_counter,y,x), float64 dic(time_counter,y,x), float64 don(time_counter,y,x), float64 mes(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float64 nh4(time_counter,y,x), float64 no3(time_counter,y,x), float64 oxy(time_counter,y,x), float64 phy(time_counter,y,x), float64 pon(time_counter,y,x), float64 sil(time_counter,y,x), float64 ta(time_counter,y,x), float64 time_counter(time_counter), float64 tur(time_counter,y,x), float64 zoo(time_counter,y,x)
    groups: 



In [17]:
# w2 = nc.Dataset('/data/tjarniko/MEOPAR/rivers-climatology/skog/rivers_skog_tracers_m12d31.nc')
# print(w2)

In [18]:
# /data/tjarniko/MEOPAR/rivers/bio
# rivers_bio_tracers_m
nclist =[]
ncblist =[]
days = [31,29,31,30,31,30,31,31,30,31,30,31]
for m in range (1,13):
    if m < 10:
        tm = '0' + str(m)
    else:
        tm = str(m)
    
    tdaynum = days[m-1]
    for d in range(1,tdaynum+1):
        if d < 10:
            td = '0' + str(d)
        else:
            td = str(d)
        tnc = 'rivers_skog_tracers_m' + tm + 'd' + td + '.nc'
        tnc2 = 'rivers_bio_tracers_m' + tm + 'd' + td + '.nc'
        nclist.append(tnc)
        ncblist.append(tnc2)
        print(tnc)


rivers_skog_tracers_m01d01.nc
rivers_skog_tracers_m01d02.nc
rivers_skog_tracers_m01d03.nc
rivers_skog_tracers_m01d04.nc
rivers_skog_tracers_m01d05.nc
rivers_skog_tracers_m01d06.nc
rivers_skog_tracers_m01d07.nc
rivers_skog_tracers_m01d08.nc
rivers_skog_tracers_m01d09.nc
rivers_skog_tracers_m01d10.nc
rivers_skog_tracers_m01d11.nc
rivers_skog_tracers_m01d12.nc
rivers_skog_tracers_m01d13.nc
rivers_skog_tracers_m01d14.nc
rivers_skog_tracers_m01d15.nc
rivers_skog_tracers_m01d16.nc
rivers_skog_tracers_m01d17.nc
rivers_skog_tracers_m01d18.nc
rivers_skog_tracers_m01d19.nc
rivers_skog_tracers_m01d20.nc
rivers_skog_tracers_m01d21.nc
rivers_skog_tracers_m01d22.nc
rivers_skog_tracers_m01d23.nc
rivers_skog_tracers_m01d24.nc
rivers_skog_tracers_m01d25.nc
rivers_skog_tracers_m01d26.nc
rivers_skog_tracers_m01d27.nc
rivers_skog_tracers_m01d28.nc
rivers_skog_tracers_m01d29.nc
rivers_skog_tracers_m01d30.nc
rivers_skog_tracers_m01d31.nc
rivers_skog_tracers_m02d01.nc
rivers_skog_tracers_m02d02.nc
rivers_sko

In [19]:
import subprocess
#twc = 'ncks -4 -L4 -O rivers_bio_tracers_m12d22.nc rivers_bio_tracers_m12d22.nc'
resdir = '/data/tjarniko/MEOPAR/rivers-climatology/skog_sens_highFras/'

for day in range(0,366):
    tc = 'ncks -4 -L4 -O ' + nclist[day] + ' ' + nclist[day]
    print(tc)
    %time subprocess.call(tc, shell=True,cwd=resdir)

ncks -4 -L4 -O rivers_skog_tracers_m01d01.nc rivers_skog_tracers_m01d01.nc
CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 608 ms
ncks -4 -L4 -O rivers_skog_tracers_m01d02.nc rivers_skog_tracers_m01d02.nc
CPU times: user 4 ms, sys: 48 ms, total: 52 ms
Wall time: 517 ms
ncks -4 -L4 -O rivers_skog_tracers_m01d03.nc rivers_skog_tracers_m01d03.nc
CPU times: user 0 ns, sys: 48 ms, total: 48 ms
Wall time: 588 ms
ncks -4 -L4 -O rivers_skog_tracers_m01d04.nc rivers_skog_tracers_m01d04.nc
CPU times: user 0 ns, sys: 60 ms, total: 60 ms
Wall time: 592 ms
ncks -4 -L4 -O rivers_skog_tracers_m01d05.nc rivers_skog_tracers_m01d05.nc
CPU times: user 0 ns, sys: 60 ms, total: 60 ms
Wall time: 609 ms
ncks -4 -L4 -O rivers_skog_tracers_m01d06.nc rivers_skog_tracers_m01d06.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 597 ms
ncks -4 -L4 -O rivers_skog_tracers_m01d07.nc rivers_skog_tracers_m01d07.nc
CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 553 ms
ncks -4 -L4 -O river

CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 541 ms
ncks -4 -L4 -O rivers_skog_tracers_m02d29.nc rivers_skog_tracers_m02d29.nc
CPU times: user 8 ms, sys: 60 ms, total: 68 ms
Wall time: 586 ms
ncks -4 -L4 -O rivers_skog_tracers_m03d01.nc rivers_skog_tracers_m03d01.nc
CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 614 ms
ncks -4 -L4 -O rivers_skog_tracers_m03d02.nc rivers_skog_tracers_m03d02.nc
CPU times: user 8 ms, sys: 60 ms, total: 68 ms
Wall time: 584 ms
ncks -4 -L4 -O rivers_skog_tracers_m03d03.nc rivers_skog_tracers_m03d03.nc
CPU times: user 8 ms, sys: 52 ms, total: 60 ms
Wall time: 601 ms
ncks -4 -L4 -O rivers_skog_tracers_m03d04.nc rivers_skog_tracers_m03d04.nc
CPU times: user 8 ms, sys: 40 ms, total: 48 ms
Wall time: 560 ms
ncks -4 -L4 -O rivers_skog_tracers_m03d05.nc rivers_skog_tracers_m03d05.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 617 ms
ncks -4 -L4 -O rivers_skog_tracers_m03d06.nc rivers_skog_tracers_m03d06.nc
CPU times: user 0 ns

CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 613 ms
ncks -4 -L4 -O rivers_skog_tracers_m04d28.nc rivers_skog_tracers_m04d28.nc
CPU times: user 4 ms, sys: 60 ms, total: 64 ms
Wall time: 594 ms
ncks -4 -L4 -O rivers_skog_tracers_m04d29.nc rivers_skog_tracers_m04d29.nc
CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 598 ms
ncks -4 -L4 -O rivers_skog_tracers_m04d30.nc rivers_skog_tracers_m04d30.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 612 ms
ncks -4 -L4 -O rivers_skog_tracers_m05d01.nc rivers_skog_tracers_m05d01.nc
CPU times: user 8 ms, sys: 52 ms, total: 60 ms
Wall time: 576 ms
ncks -4 -L4 -O rivers_skog_tracers_m05d02.nc rivers_skog_tracers_m05d02.nc
CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 614 ms
ncks -4 -L4 -O rivers_skog_tracers_m05d03.nc rivers_skog_tracers_m05d03.nc
CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 616 ms
ncks -4 -L4 -O rivers_skog_tracers_m05d04.nc rivers_skog_tracers_m05d04.nc
CPU times: user 0 ns

CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 587 ms
ncks -4 -L4 -O rivers_skog_tracers_m06d26.nc rivers_skog_tracers_m06d26.nc
CPU times: user 8 ms, sys: 60 ms, total: 68 ms
Wall time: 615 ms
ncks -4 -L4 -O rivers_skog_tracers_m06d27.nc rivers_skog_tracers_m06d27.nc
CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 610 ms
ncks -4 -L4 -O rivers_skog_tracers_m06d28.nc rivers_skog_tracers_m06d28.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 550 ms
ncks -4 -L4 -O rivers_skog_tracers_m06d29.nc rivers_skog_tracers_m06d29.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 603 ms
ncks -4 -L4 -O rivers_skog_tracers_m06d30.nc rivers_skog_tracers_m06d30.nc
CPU times: user 4 ms, sys: 40 ms, total: 44 ms
Wall time: 589 ms
ncks -4 -L4 -O rivers_skog_tracers_m07d01.nc rivers_skog_tracers_m07d01.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 601 ms
ncks -4 -L4 -O rivers_skog_tracers_m07d02.nc rivers_skog_tracers_m07d02.nc
CPU times: user 0 ns

CPU times: user 0 ns, sys: 52 ms, total: 52 ms
Wall time: 603 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d24.nc rivers_skog_tracers_m08d24.nc
CPU times: user 0 ns, sys: 44 ms, total: 44 ms
Wall time: 585 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d25.nc rivers_skog_tracers_m08d25.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 615 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d26.nc rivers_skog_tracers_m08d26.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 600 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d27.nc rivers_skog_tracers_m08d27.nc
CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 540 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d28.nc rivers_skog_tracers_m08d28.nc
CPU times: user 4 ms, sys: 64 ms, total: 68 ms
Wall time: 601 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d29.nc rivers_skog_tracers_m08d29.nc
CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 563 ms
ncks -4 -L4 -O rivers_skog_tracers_m08d30.nc rivers_skog_tracers_m08d30.nc
CPU times: user 8 ms

CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 578 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d22.nc rivers_skog_tracers_m10d22.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 546 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d23.nc rivers_skog_tracers_m10d23.nc
CPU times: user 4 ms, sys: 60 ms, total: 64 ms
Wall time: 612 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d24.nc rivers_skog_tracers_m10d24.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 607 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d25.nc rivers_skog_tracers_m10d25.nc
CPU times: user 4 ms, sys: 48 ms, total: 52 ms
Wall time: 588 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d26.nc rivers_skog_tracers_m10d26.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 588 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d27.nc rivers_skog_tracers_m10d27.nc
CPU times: user 8 ms, sys: 60 ms, total: 68 ms
Wall time: 615 ms
ncks -4 -L4 -O rivers_skog_tracers_m10d28.nc rivers_skog_tracers_m10d28.nc
CPU times: user 8 ms

CPU times: user 4 ms, sys: 56 ms, total: 60 ms
Wall time: 594 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d20.nc rivers_skog_tracers_m12d20.nc
CPU times: user 0 ns, sys: 44 ms, total: 44 ms
Wall time: 589 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d21.nc rivers_skog_tracers_m12d21.nc
CPU times: user 0 ns, sys: 68 ms, total: 68 ms
Wall time: 613 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d22.nc rivers_skog_tracers_m12d22.nc
CPU times: user 0 ns, sys: 44 ms, total: 44 ms
Wall time: 579 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d23.nc rivers_skog_tracers_m12d23.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 611 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d24.nc rivers_skog_tracers_m12d24.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 552 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d25.nc rivers_skog_tracers_m12d25.nc
CPU times: user 0 ns, sys: 64 ms, total: 64 ms
Wall time: 596 ms
ncks -4 -L4 -O rivers_skog_tracers_m12d26.nc rivers_skog_tracers_m12d26.nc
CPU times: user 4 ms